In [26]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.4 MB/s eta 0:00:00
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached fosforml-1.1.6-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached matplotlib-3.9.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scikit_learn-1.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using

  Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached pyOpenSSL-24.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached PyJWT-2.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached platformdirs-4.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached tomlkit-0.13.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
  Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached botocore-1.34.131-py3-none-any.whl.metadata (5.7 kB)
  Using cached wrapt-1.16.0-cp39-cp39-manylinux_2_5_x86_64.

Using cached aiobotocore-2.13.1-py3-none-any.whl (76 kB)
Using cached aiohttp-3.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached cffi-1.16.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (443 kB)
Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl (3.9 MB)
Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached platformdirs-4.2.2-py3-none-any.whl (18 kB)
Using cached PyJWT-2.9.0-py3-none-any.whl (22 kB)
Using cached pyOpenSSL-24.2.1-py3-none-any.whl (58 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached setuptools-72.1.0-py3-none-any.whl (2.3 MB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Found existing installation: urllib3 1.26.15
Uninstalling urllib3-1.26.15:
  Successfully uninstalled urllib3-1.26.15
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached urllib3-1.26.15-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
refractml 1.0.8 requires PyYAML==6.0, but you have pyyaml 6.0.1 which is incompatible.
refractml 1.0.8 requires urllib3==1.26.19, but you have urllib3 1.26.15 which is incompatible.
snowflake-ml-python 1.5.0 r

  Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached pyOpenSSL-24.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached PyJWT-2.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached platformdirs-4.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached tomlkit-0.13.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
  Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached botocore-1.34.131-py3-none-any.whl.metadata (5.7 kB)
  Using cached wrapt-1.16.0-cp39-cp39-manylinux_2_5_x86_64.

Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached xgboost-1.7.6-py3-none-manylinux2014_x86_64.whl (200.3 MB)
Using cached pyarrow-17.0.0-cp39-cp39-manylinux_2_28_x86_64.whl (39.9 MB)
Using cached aiobotocore-2.13.1-py3-none-any.whl (76 kB)
Using cached aiohttp-3.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached cffi-1.16.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (443 kB)
Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl (3.9 MB)
Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached platformdirs-4.2.2-py3-none-any.whl (18 kB)
Using cached PyJWT-2.9.0-py3-none-any.whl (22 kB)
Using cached pyOpenSSL-24.2.1-py3-none-any.

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached fosforio-1.0.2-py3-none-any.whl.metadata (13 kB)
  Using cached pandas-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached fosforio-1.0.2-py3-none-any.whl (20 kB)
Using cached pandas-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
Using cached python_dateutil-2

  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached refractio-2.1.5.6-py3-none-any.whl (20 kB)
Using cached pandas-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.1.6 requires cloudpickle==2.2.1, but you have cloudpickle 1.6.0 which is incompatible.
fosforml 1.1.6 requires scikit-learn==1.3.2, but you have scikit-learn 1.2.1 which is incompatible.
refractio 2.1.5.6 requires pandas==2.0.0, but you have pandas 1.5.3 which is incompatible.
snowflake-ml-python 1.5.0 requires cloudpickle>=2.0.0, but you have cloudpickle 1.6.0 which is incompatible.
jupyterlab 3.2.4 requires jupyter-server~=1.4, but you have jupyter-serve

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached xgboost-2.1.1-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached dice_ml-0.11-py3-none-any.whl.metadata (20 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scikit_learn-1.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached pandas_profiling-3.6.6-py2.py3-none-any.whl.metadata (4.5 kB)
  Using cached plotly-5.23.0-py3-none-any.whl.metada

  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached importlib_metadata-8.2.0-py3-none-any.whl.metadata (4.7 kB)
INFO: pip is looking at multiple versions of visions to determine which version is compatible with other requirements. This could take a while.
  Using cached visions-0.7.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached tangled_up_in_unicode-0.2.0-py3-none-any.whl.metadata (4.8 kB)
INFO: pip is looking at multiple versions of visions[type-image-path] to determine which version is compatible with other requirements. This could take a while.
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
Using cached xgboost-2.1.1-py3-none-manylinux_2_28_x86_64.whl (153.9 MB)
Usi

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [9]:
from refractio import snowflake
snowflake.get_connection(connection_name="HR_ATTRITION")

Exception occurred in getting snowflake connection: 'connectionSources'


In [10]:
df = snowflake.get_dataframe("HR_ATTRITION")
df

,SALARY,EMPLOYEE_ID,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN_STATUS_TABLE,CHURN_VALUE_TABLE,CITY,...,RETENTION,SUM_OF_TENURE,SUM_OF_AGE,JOB_STARTDATE,JOB_ENDDATE,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,USER_ID
0,43118.65,94215814,High,20,Outstanding,No,Medium,FALSE,0,Charlotte,...,0,0,0,2016-06-01,2019-08-01,4,Best,Travel Rarely,Medium,334493477
1,40808.18,72815065,Very High,12,Low,No,Very High,TRUE,1,Charlotte,...,0,0,0,2015-05-01,None,7,Best,Travel Frequently,Low,713161033
2,36394.69,14265310,Low,13,Good,No,Medium,FALSE,0,Pittsburgh,...,0,0,0,2017-06-01,2018-07-01,8,Good,Travel Rarely,High,154131612
3,96880.66,38467325,Low,21,Good,No,Very High,FALSE,0,Dallas,...,0,0,0,2019-09-01,2024-07-01,14,Bad,Travel Rarely,Very High,518711452
4,97579.66,98539130,High,17,Good,No,Low,FALSE,0,Dallas,...,0,0,0,2019-09-01,2024-08-01,9,Best,Travel Frequently,Low,518711452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,101713.50,66752547,Low,17,Good,No,Very High,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-06-01,1,Good,Travel Rarely,Low,492160448
299996,101640.50,61579487,High,13,Excellent,No,Very High,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-06-01,1,Best,Travel Frequently,Low,492160448
299997,101998.50,48056583,Very High,21,Outstanding,No,Medium,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-06-01,1,Good,Travel Rarely,Low,492160448
299998,101306.50,86413761,Medium,20,Low,No,Low,FALSE,0,Houston,...,609,37548,9563,2021-12-01,2023-06-01,1,Best,Travel Frequently,Medium,492160448


In [11]:
print(df.isnull().sum())

SALARY                                             0
EMPLOYEE_ID                                        0
JOB_SATISFACTION                                   0
PERCENTAGE_SALARY_HIKE                             0
PERFORMANCE_RATING                                 0
OVER_TIME                                          0
RELATIONSHIP_SATISFACTION                          0
CHURN_STATUS_TABLE                                 0
CHURN_VALUE_TABLE                                  0
CITY                                               0
DISTANCE                                           0
DEGREE_CLEAN                                       0
ETHNICITY                                          0
MARITAL_STATUS                                     0
ROLE                                               0
COMPANY                                            0
ORGANIZATION_TYPE                                  0
ORGANIZATION_OWNERSHIP                             0
STATE                                         

In [12]:
Original_df = df.dropna()

In [13]:
print(Original_df.isnull().sum())

SALARY                                         0
EMPLOYEE_ID                                    0
JOB_SATISFACTION                               0
PERCENTAGE_SALARY_HIKE                         0
PERFORMANCE_RATING                             0
OVER_TIME                                      0
RELATIONSHIP_SATISFACTION                      0
CHURN_STATUS_TABLE                             0
CHURN_VALUE_TABLE                              0
CITY                                           0
DISTANCE                                       0
DEGREE_CLEAN                                   0
ETHNICITY                                      0
MARITAL_STATUS                                 0
ROLE                                           0
COMPANY                                        0
ORGANIZATION_TYPE                              0
ORGANIZATION_OWNERSHIP                         0
STATE                                          0
COUNTRY                                        0
GENDER              

In [15]:
df = Original_df.drop(["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","RETENTION","SUM_OF_TENURE","SUM_OF_AGE"], axis = 1)

In [22]:
CATEGORICAL_COLUMNS = ["ROLE","GENDER", "ETHNICITY","ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP","COMPANY","CITY","STATE","COUNTRY","DISTANCE","CHURN_STATUS_TABLE",
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARITAL_STATUS","OVER_TIME","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE","OVERTIME_HOURS","PERCENTAGE_SALARY_HIKE"]
LABEL_COLUMNS = ["CHURN_VALUE_TABLE "]
DROPPED_COLUMNS = ["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","RETENTION","SUM_OF_TENURE","SUM_OF_AGE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [23]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS
feature_columns = [col for col in feature_columns if col in Original_df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in Original_df.columns]
 
# Split data into features and labels
X = Original_df[feature_columns + DROPPED_COLUMNS]
y = Original_df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [209924, 0]